# Quickstart

Via AgentChat, you can build applications quickly using preset agents.
To illustrate this, we will begin with creating a team of a single tool-use
agent that you can chat with.

The following code uses the OpenAI model. If you haven't already, you need to
install the following package and extension:

In [1]:
%%capture
# capture magic suppresses install output
!poetry add "autogen-agentchat" "autogen-ext[openai,azure]"

To use Azure OpenAI models and AAD authentication,
you can follow the instructions [here](./tutorial/models.ipynb#azure-openai).

In [2]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient


# Define a tool
async def get_weather(city: str) -> str:
    return f"The weather in {city} is 73 degrees and Sunny."


async def main() -> None:
    # Define an agent
    weather_agent = AssistantAgent(
        name="weather_agent",
        model_client=OpenAIChatCompletionClient(
            model="gpt-4o-2024-08-06",
            # api_key="YOUR_API_KEY",
        ),
        tools=[get_weather],
    )

    # Define a team with a single agent and maximum auto-gen turns of 1.
    agent_team = RoundRobinGroupChat([weather_agent], max_turns=1)

    while True:
        # Get user input from the console.
        user_input = input("Enter a message (type 'exit' to leave): ")
        if user_input.strip().lower() == "exit":
            break
        # Run the team and stream messages to the console.
        stream = agent_team.run_stream(task=user_input)
        await Console(stream)


# NOTE: if running this inside a Python script you'll need to use asyncio.run(main()).
await main()

Enter a message (type 'exit' to leave):  how is the weather in boston?


---------- user ----------
how is the weather in boston?
---------- weather_agent ----------
[FunctionCall(id='call_ook5gp87oLVrYBpTK4kYEXRx', arguments='{"city":"boston"}', name='get_weather')]
---------- weather_agent ----------
[FunctionExecutionResult(content='The weather in boston is 73 degrees and Sunny.', call_id='call_ook5gp87oLVrYBpTK4kYEXRx')]
---------- weather_agent ----------
The weather in boston is 73 degrees and Sunny.


Enter a message (type 'exit' to leave):  exit


The code snippet above introduces two high level concepts in AgentChat: *Agent* and *Team*.  An Agent helps us define what actions are taken when a message is received. Specifically, we use the {py:class}`~autogen_agentchat.agents.AssistantAgent` preset - an agent that can be given access to a model (e.g., LLM) and tools (functions) that it can then use to address tasks.  A Team helps us define the rules for how agents interact with each other.  In the  {py:class}`~autogen_agentchat.teams.RoundRobinGroupChat` team, agents respond in a sequential round-robin fashion.
In this case, we have a single agent, so the same agent is used for each round.

## What's Next?

Now that you have a basic understanding of how to define an agent and a team, consider following the [tutorial](./tutorial/models) for a walkthrough on other features of AgentChat.

